flag{ConGrAts-YOUr-PAYlOaD-bEaTs-sHannOn}

flag{the-WHEElS-ThaT-SINg-AN-UneNDing-DReAm}

In [10]:
import random
from typing import Literal


def encode_huffman(num: int) -> tuple[int, Literal[7, 8, 9]]:
    """
    Encode the Huffman code, returns encoded bits and bit length
    """
    # Use the fixed Huffman table of DEFLATE
    assert 0 <= num <= 287
    if num <= 143:
        # 8 bits, first section
        return (num + 0b0011_0000, 8)
    elif num <= 255:
        # 9 bits
        return (num - 144 + 0b1100_1000_0, 9)
    elif num <= 279:
        # 7 bits
        return (num - 256, 7)
    else:
        # 8 bits, second section
        return (num - 280 + 0b1100_0000, 8)


def decode_huffman(encoded_bits: int) -> tuple[int, Literal[7, 8, 9]]:
    """
    Decode the Huffman code (9 bits), returns decoded result and bit length
    """
    # Use the fixed Huffman table of DEFLATE
    # encoded_bits contains 9 bits
    if encoded_bits <= 0b0010_1110_0:
        # 7 bits
        encoded_bits >>= 2
        return (encoded_bits + 256, 7)
    elif 0b0011_0000_0 <= encoded_bits <= 0b1011_1111_0:
        # 8 bits, first section
        encoded_bits >>= 1
        return (encoded_bits - 0b0011_0000 + 0, 8)
    elif 0b1100_0000_0 <= encoded_bits <= 0b1100_0111_0:
        # 8 bits, second section
        encoded_bits >>= 1
        return (encoded_bits - 0b1100_0000 + 280, 8)
    else:
        # 9 bits
        return (encoded_bits - 0b1100_1000_0 + 144, 9)

def reverse_bit_int(num: int, bit_length: int = 8) -> int:
    """
    Reverse the bits of an integer
    """
    res = 0
    for _ in range(bit_length):
        res <<= 1
        res |= num & 1
        num >>= 1
    return res


def reverse_bits(bytestream: bytes) -> bytes:
    """
    Reverse the bits of the bytestream
    """
    return bytes(reverse_bit_int(x) for x in bytestream)

In [11]:
class GzipParser:
    def __init__(self, bytestream: bytes):
        self.bytestream = bytestream
        self.pos = (0, 0)  # (byte, bit); maintain bit < 8
        self.block_no = 0
        self.output_buf = bytearray()
        self.indent_level = 0

    def log(self, msg: str):
        print("  " * self.indent_level + msg)

    def indent(self):
        self.indent_level += 1

    def dedent(self):
        self.indent_level -= 1

    def parse(self):
        while self.pos[0] < len(self.bytestream):
            last_block = self.parse_block()
            if last_block:
                break

    def peek_bits(self, n: int = 1, is_move_cursor: bool = False) -> int:
        byte, bit = self.pos
        res = 0
        for _ in range(n):
            res <<= 1
            # LSB first
            res |= int(bool(self.bytestream[byte] & (1 << bit)))
            bit += 1
            if bit == 8:
                bit = 0
                byte += 1
        # print(f"peek {n} bits, cur first 2 bytes {self.bytestream[byte]:08b} {self.bytestream[byte+1]:08b} -> {res:0{n}b}, pos {self.pos}")
        if is_move_cursor:
            self.pos = (byte, bit)
        return res

    def read_bits(self, n: int = 1):
        return self.peek_bits(n, True)
    
    def read_bits_rev(self, n: int = 1):
        return reverse_bit_int(self.read_bits(n), n)

    def advance(self, n: int):
        byte, bit = self.pos
        byte += n // 8
        bit += n % 8
        byte += bit // 8
        bit %= 8
        self.pos = (byte, bit)

    def parse_block(self):
        # Read the block header
        is_last = self.read_bits()
        block_type = reverse_bit_int(self.read_bits(2), 2)
        assert block_type == 0b01, "Only support fixed Huffman code"
        # Parse the block
        self.log(f"{is_last:01b} {reverse_bit_int(block_type, 2):02b}: Block {self.block_no}")
        self.indent()
        while True:
            is_fin = self.parse_huffman_code()
            if is_fin:
                break
        self.dedent()
        self.block_no += 1
        self.advance(8 - self.pos[1])
        return is_last

    def parse_huffman_code(self):
        code = self.peek_bits(9)
        decoded, bit_length = decode_huffman(code)
        assert 0 <= decoded <= 285
        str_huffman = f"{(code >> (9 - bit_length)):0{bit_length}b}"
        self.advance(bit_length)
        if decoded == 256:
            self.log(f"{str_huffman}: $$")
            return True
        if decoded < 256:
            self.log(f"{str_huffman}: [{bytes([decoded])}]")
            self.output_buf.append(decoded)
            return False
        length, extra_str = self.parse_length(decoded)
        distance, distance_str = self.parse_distance()
        cur_pos = len(self.output_buf)
        # memcpy
        copy_from = cur_pos - distance
        for _ in range(length):
            self.output_buf.append(self.output_buf[copy_from])
            copy_from += 1
        self.log(f"{str_huffman}{extra_str} {distance_str}: ({length}, {distance}) [{bytes(self.output_buf[cur_pos:cur_pos+length])}]")
        return False

    def parse_length(self, code: int) -> tuple[int, str]:
        maps = {257: (0, 3), 267: (1, 15), 277: (4, 67), 258: (0, 4), 268: (1, 17), 278: (4, 83), 259: (0, 5), 269: (2, 19), 279: (4, 99), 260: (0, 6), 270: (2, 23), 280: (4, 115), 261: (0, 7), 271: (2, 27), 281: (
            5, 131), 262: (0, 8), 272: (2, 31), 282: (5, 163), 263: (0, 9), 273: (3, 35), 283: (5, 195), 264: (0, 10), 274: (3, 43), 284: (5, 227), 265: (1, 11), 275: (3, 51), 285: (0, 258), 266: (1, 13), 276: (3, 59)}
        extra_bits, base_length = maps[code]
        extra_bits_read = self.read_bits(extra_bits)
        extra_bits_str = f"-{extra_bits_read:0{extra_bits}b}" if extra_bits > 0 else ""
        length = base_length + reverse_bit_int(extra_bits_read, extra_bits)
        return (length, extra_bits_str)

    def parse_distance(self) -> tuple[int, str]:
        maps = {0: (0, 1), 10: (4, 33), 20: (9, 1025), 1: (0, 2), 11: (4, 49), 21: (9, 1537), 2: (0, 3), 12: (5, 65), 22: (10, 2049), 3: (0, 4), 13: (5, 97), 23: (10, 3073), 4: (1, 5), 14: (6, 129), 24: (11, 4097), 5: (
            1, 7), 15: (6, 193), 25: (11, 6145), 6: (2, 9), 16: (7, 257), 26: (12, 8193), 7: (2, 13), 17: (7, 385), 27: (12, 12289), 8: (3, 17), 18: (8, 513), 28: (13, 16385), 9: (3, 25), 19: (8, 769), 29: (13, 24577)}
        code = self.read_bits(5)
        extra_bits, base_distance = maps[code]
        extra_bits_read = self.read_bits(extra_bits)
        distance_str = f"{code:05b}-{extra_bits_read:0{extra_bits}b}" if extra_bits > 0 else f"{code:05b}"
        distance = base_distance + reverse_bit_int(extra_bits_read, extra_bits)
        return (distance, distance_str)

In [12]:
# # Deprecated (attempted to use static Huffman tree)
# ADOPTED = 18
# lightweight_code_base = [0b01000000, 0b10000000]
# lightweight_code = [x | (1 << i) for x in lightweight_code_base for i in range(6)] + lightweight_code_base
# filtered_lw_code = [x for x in lightweight_code if 0x20 <= (decode_huffman(x << 1)[0] ^ 13) <= 0x7e]
# assert len([x for x in filtered_lw_code if x.bit_count() > 2]) == 0
# mw_code = [((1 << i) | x) for x in lightweight_code for i in range(6) if ((1 << i) | x) not in lightweight_code]
# filtered_mw_code = list(set([x for x in mw_code if 0x20 <= (decode_huffman(x << 1)[0] ^ 13) <= 0x7e]))[:ADOPTED - len(filtered_lw_code)]
# assert len([x for x in filtered_mw_code if x.bit_count() == 3]) == len(filtered_mw_code)
# filtered_code = filtered_lw_code + filtered_mw_code
# assert len(set(filtered_code)) == ADOPTED
# assert all(0x20 <= (decode_huffman(x << 1)[0] ^ 13) <= 0x7e for x in filtered_code)
# assert all(x.bit_count() <= 3 for x in filtered_code)

In [13]:
# print(bytes(map(lambda x: decode_huffman(x << 1)[0] ^ 13, filtered_code)).decode())
# charset = [decode_huffman(x << 1)[0] ^ 13 for x in filtered_code]
# def get_balanced_code(charset: list[int]) -> bytearray:
#     res = bytearray()
#     target_len = ADOPTED * ADOPTED
#     visited_from = [[False] * ADOPTED for _ in range(ADOPTED)]
#     cur = 0
#     res.append(charset[cur])
#     # https://math.stackexchange.com/questions/4926360/finding-an-eulerian-path-on-complete-graphs
#     for _ in range(target_len):
#         next = -1
#         for i in range(cur, ADOPTED):
#             if not visited_from[cur][i]:
#                 next = i
#                 break
#         else:
#             for i in range(cur):
#                 if not visited_from[cur][i]:
#                     next = i
#                     break
#         assert next != -1
#         visited_from[cur][next] = True
#         cur = next
#         res.append(charset[cur])
#     assert len(res) == target_len + 1
#     return res
# flag_1_bytestream = get_balanced_code(charset)
# new_charset = [x for x in range(0x20, 0x7e) if x not in charset]
# flag_1_bytestream.extend(get_balanced_code(new_charset))
# print(flag_1_bytestream.decode()[:1000])


In [14]:
def reverse_shuffle(seed, string):
    random.seed(seed)
    ids = list(range(len(string)))
    random.shuffle(ids)
    # find the inverse of permutation ids
    inv_ids = [0] * len(ids)
    for i, x in enumerate(ids):
        inv_ids[x] = i
    for i in range(len(ids)):
        assert ids[inv_ids[i]] == i
    return "".join(string[i] for i in inv_ids)

In [15]:
num_chars = 16
target_block_size = 256
sel_start = 0x30 # 0x30 - 0x4f
res = bytearray()
all_chars = [61, 60, 63, 57, 53, 77, 92, 62, 56, 59, 52, 55, 49, 76, 79, 73, 69, 95, 94, 88, 84, 108, 58, 54, 48, 51, 78, 72, 75, 68, 71, 65, 89, 91, 90, 87, 86, 80, 111, 110, 104, 100, 50, 74, 70, 64, 66, 85, 81, 83, 82, 105, 107, 106, 103, 102, 96, 93, 109, 101, 97, 99, 98, 67]
for i in range(4):
    cur_range = all_chars[i * num_chars:(i + 1) * num_chars]
    cur_chars = cur_range
    visited_from = [[False] * num_chars for _ in range(num_chars)]
    # for i in range(num_chars):
    #     visited_from[i][i] = True
    cur = 0
    tres = bytearray()
    # tres.append(cur_chars[cur])
    tres.append(cur_chars[cur])
    if i > 0:
        cur_target_block_size = ((1024 - target_block_size + 3) // 4 + 1) // 4 * 4
    else:
        cur_target_block_size = target_block_size
    # https://math.stackexchange.com/questions/4926360/finding-an-eulerian-path-on-complete-graphs
    for _ in range(0, cur_target_block_size - 1):
        next = -1
        for i in range(cur, num_chars):
            if not visited_from[cur][i]:
                next = i
                break
        else:
            for i in range(cur):
                if not visited_from[cur][i]:
                    next = i
                    break
        assert next != -1
        visited_from[cur][next] = True
        cur = next
        # tres.append(cur_chars[cur])
        tres.append(cur_chars[cur])
    res.extend(tres[:250])
    # res.extend(reversed(tres))
print(reverse_shuffle("mamba_out", res.decode()))
# print(len(res))
flag_1_unshuffled_res = res

XSXc1Ke@R8A21aN3_NQL7W\9D^`EdEj]J;g4?624S7AicenPOe2P<\Nf_`>LP8=_D9VaVI5>EjgdT8BNenM>]dc;f=HBLm<4Zb1;AB;F3^>Yhl2T:4L`H<g]9P`UQKHW;\Z>T5nUSY3[25g=Jl>FGg^C;Dj38Wlmb:TL\O6NHE\I`I0nLDNPYIN8L?ETR9?JM9G<cPO5Hf9Hc7o:mR5HWVE?5mmU2h0iaC1XZWbnOLC;]i<@]=Y31f=L47eoah][Elbb;;hekU=fLJBVXZ3`j1Xc^kVmc_>GcKFMSQ5C_D>MRIY3=fZo1eI\WO@OlcS5G5892H=WSHkQEC6:[ILQG`Ch<?bFM4Ic@<0NWkeDX[1[lnN1IR5FH=mBTD?=<[S?Co7cC3PTE?ojjAAIAR=dT@U7<68JK>WoKO7X=kLI[;<BMg>LnQgoo1]Sa1@=iG8<6GaDU83526ImOJYkfg\;AC0]_6b=J<:BR0]RGU7UbJ_idf1>`U9F:^And9Ya4[\Zhl]:YW^9JHZ?_=jQMiOC^J0mL9:>70nT5V2d4?g^48Bj1Yf477K]44Bdi;dOhP]lb?cEGieJ[:KRoPbfEh@QPMVWQ5BnBo88a4<@idk27fkekgQIO7MD9Rekg4TKURQXX2jD^K>lYLm98_S0iA2<;>h>?A6ZF9ni[F3b`eVMZE[LK7D\Z06_ZMS^`@oh\X7ARd5n:[hI;mM_\Vj4d;T3OXa6<8bV\6@@FhfaZV@5mQA_M:1?^YUBGOFkPM=\8HYOlC:JC3UO0WlKa1PIjS?FIO46XgDK^oS0V`i\Fj1;9G87aGl>N0kNMN\`T?


In [16]:
byte_seq = b"[What can I say? Mamba out! --KobeBryant] "
# bitseq_to_char = {0: 48, 1: 49, 2: 50, 3: 51, 4: 52, 5: 53, 6: 54, 7: 55, 8: 56, 9: 57, 10: 58, 11: 59, 12: 60, 13: 61, 14: 62, 15: 63, 16: 64, 17: 65, 18: 66, 19: 67, 20: 68, 21: 69, 22: 70, 23: 71, 24: 72, 25: 73, 26: 74, 27: 75, 28: 76, 29: 77, 126: 78, 30: 79, 31: 80, 32: 81, 33: 82, 34: 83, 35: 84, 36: 85, 37: 86, 38: 87, 39: 88, 40: 89, 41: 90, 42: 91, 43: 92, 44: 93, 45: 94, 46: 95, 47: 96, 48: 97, 49: 98, 50: 99, 51: 100, 52: 101, 53: 102, 54: 103, 55: 104, 56: 105, 57: 106, 58: 107, 59: 108, 60: 109, 61: 110, 62: 111}
# 18, 7
bitseq_to_char = {0: 48, 1: 49, 2: 50, 3: 51, 4: 52, 5: 53, 6: 54, 7: 55, 8: 56, 9: 57, 10: 58, 11: 59, 12: 60, 13: 61, 14: 62, 15: 63, 16: 64, 17: 65, 18: 66, 19: 67, 20: 68, 21: 69, 22: 70, 23: 71, 24: 72, 25: 73, 26: 74, 27: 75, 28: 76, 29: 77, 30: 78, 126: 79, 31: 80, 32: 81, 33: 82, 34: 83, 35: 84, 36: 85, 37: 86, 38: 87, 39: 88, 40: 89, 41: 90, 42: 91, 43: 92, 44: 93, 45: 94, 46: 95, 47: 96, 48: 97, 49: 98, 50: 99, 51: 100, 52: 101, 53: 102, 54: 103, 55: 104, 56: 105, 57: 106, 58: 107, 59: 108, 60: 109, 61: 110, 62: 111}

num_chars = len(all_chars)
parser = GzipParser(byte_seq)
sts = set()
cur = parser.read_bits(6)
char_seq = bytearray([bitseq_to_char[cur] ^ 13])
for i in range(54):
    last = cur
    cur = parser.read_bits(6)
    if cur == 0b111111:
        cur = cur << 1 | parser.read_bits(1)
    if (last, cur) not in sts:
        sts.add((last, cur))
    char_seq.append(bitseq_to_char[cur] ^ 13)
copied_flag_1_res = flag_1_unshuffled_res.copy()
for i in char_seq:
    # find one i in copied_flag_1_res, remove it
    copied_flag_1_res.remove(i)
copied_flag_1_res.remove(79 ^ 13)
char_seq.insert(0, 79 ^ 13)
char_seq.insert(1, copied_flag_1_res[0])
char_seq.insert(2, copied_flag_1_res[1])
char_seq.insert(3, copied_flag_1_res[2])
char_seq.extend(copied_flag_1_res[3:])
# print(flag_1_unshuffled_res.decode())
# print(copied_flag_1_res.decode())
flag_2_unshuffled_res = char_seq
print(reverse_shuffle("mamba_out", flag_2_unshuffled_res.decode()))
# xored = [x ^ 13 for x in char_seq]
# print(bytes(xored).decode())
# start = cur
# visited_from = [[False] * num_chars for _ in range(num_chars)]

EBEa?_e2RLld1a3G03WLOAo;KD`EoOj]T=gIjNh1SG0iceoU>eJn<9Gf5f\MY<8AN1oaJ08Mljg[:52?eZ94`0a>f8K@[m\LJb1OKYOP\X1VW0XGA4?`l1g]KW`nQH=ZWOY5N^UVSh0VP5g7W;DoXkXb7:j8JUlmc\1ISL=^E07>`7HPIEAZ@I^5\<GlRL\2>T9=cV4\6fMTcMP_mR_DBD:=8]m@oF4iaCh:lJbWM8b=]i7l]=VD2g>9OCe@a[]YN1bb<9FekZ8f>3o2HHD`j1AcXQZmcA<>c3NISQ\C^NI;U<hX8fZ:Oe7?ULF4EcS90;95GI<Bd_kQ3C::YnnQ^`C^N8bd;iLcY?_5ZkmKMd?@DU65MR4dT?mnI>\1>YS=CBMcCE_NXiYjS<l0:RBh6@FL\_4J7WUWE\7X=k4M:99o7kj[oQgHh?`Sa=F?P6M?^_a324EUdO;m9ookfg;>HCH]<EbLWI9FRT]V_n?@b23iWfOMfBLAKNTK[MKa5U4PPA]Th[_IhlY76=jQL2LCO2Gm4=39C1n?5FZJT8gX;JWj<Yf1\;D]8>J@i@2?6V]XbRc>MiedJ60RV6cg7[PQ@9[3Q7FUZn<9e7IniDk[8fQekgQI>5=l1Rekg?HGBRhTG2jGAD;0^8m=<HS6i:Z<<IU;\TDPZKViEF6b`mZP@KF48J;InHE:2OSL`PBdO8MIRV4_HdU>5][07Fj4hOA6K^aGI5bX?NJn[BfadFh;mQ3K=7B57hn[N5Vk@d8;<XY4DCAWC6B94YHTa1VOjSMB1\;^:gAL^PSlo`iTdRO>\G1>e3_L^lk3=;L`NO


In [17]:
import gzip

FLAG1 = 'flag{CRYCHIC}'
FLAG2 = 'flag{MyGO!!!!!}'

def average_bit_count(s):
    return sum(c.bit_count() for c in s) / len(s)

def main():
    text = input('Input text: ')
    assert len(text)<=1000
    assert all(0x20<=ord(c)<=0x7e for c in text)

    text = [ord(c) ^ 13 for c in text]
    # text = [ord(c) for c in text]
    random.seed('mamba_out')
    random.shuffle(text)
    
    text = gzip.compress(bytes(text))
    print('\nAfter processing:\n')
    print(text[10:-8].hex())
    print(len(text))
    
    # prefix = text
    prefix = (text + b'\xFF'*256)[:256]
    print(average_bit_count(prefix))
    if average_bit_count(prefix) < 2.5:
        print('\nGood! Flag 1: ', FLAG1)
    
    if b'[What can I say? Mamba out! --KobeBryant]' in text:
        print('\nGood! Flag 2: ', FLAG2)

try:
    main()
except Exception as e:
    print('Error:', type(e))


After processing:

05c14302c3000000b02fd5b6bddab6fbffdb1200004108421082108420806114c53092a4288611454952140022029464240011608c121510093046810494924002264580401940c04450202528201504a614029310548460060c282060205422480016114c1128109604062141540944828230000530108348884228822144410ca44081151424114690608024241462028012140c1161900a4006864414913042210580410940c40450240348226144a160029428548014060b104062483420204011290c161050914486223114368c308ca224298a61c0719a66598e936555d534d3b42cc731a2026765cd329281e654d309139cd39c6860552b2c68d9340a563306ce0c07d98a70d94968d529382b61cd88d6425c35702d62ad42366833e19c410d696bd01239641ddc2cd4883358830bb9488ed4442db4c21c4cdcc21dda614339d1068b36e4c262230d37d4c1e112930e553cd4e8c864138b2b1c7988e334cdb2b2acaaa619c7695a96e3e0795db76dd7fdfdf23cceaa71e1ed5f5c36d3a1bb795a8e879d67cda2ffd26ae2ddb85aecb839dcb4e17fd9c8e7e5a4e7d5f12b17379bec74d4e3d1ce965fc5c7935b1e79a3a7d3afb14b3e5df2d1adf4ec8897f8488f8ccff452afecca6ddcf137e653bea47c6937bf29e6abdf92d963ac37f951ba53aa8fa93d65ee

In [18]:
# This is used to study the dynamic Huffman code

core_bytes = bytes.fromhex("05c14302c3000000b02fd5b6bddab6fbffdb1270eb0a2d8c0e642ccc0d2409642eac9f07a429ac4d2c0ce4ad8e2e04a4a565f99b5899905c5e981b5d17a02040318ca42886114549510028404946021001c6285101910063144840290924605204089401044c04055282025241604a21300941450866c080020206422582046011c1148102614960101244954024280803500003318884288422184214c4400a1458414112610409064842422126002841c1101106a90064604844110923145200189400444c00453280241246140a26408942054861b000012486440302021491c260010115496428124361c308a32419061ca7698e9355d5b41cc7887056b68c64a039d574c204e734271a58d50a0b5a368d82d58c8133c341b6225c76125a750ace4a5833a2b510570d5c8b58ab900dda4c38675043da1ab4440e5907370b35e20cd6e0422e922335510bad300713b7708776d8504eb4c1a20db9b0d848c30d7570b8c4a443150f353a32d9c4e20a471ee238cbaaaa199783b76d378fb36a5c78fb1797cd74b879391e769e358bfe4bab8977e36ab1e3e670d386ff65239f97939e57c7af5cdc6cb2d3518f473b5b7e151f4f6e79e48d9e4ebfc62ef974c947b7d2b3235ee2233d323ed34bbdb22bb771c7df984ff992f2a5ddfca698af7e4b668fb1dee447e94ea93ea6f694b94bf93baa9c6feabaedd6f53cafeb797cbfeff7fdbedff7fbbcba5bcfa7bfbf7ade2e7f7fbd763efddbebb667ffdaf5eadf7a7dee7af3df76ebbfeeecbdf9dabdd5ffe6e7edaebb3df7fabcbbe75bfbfa7a67dfdbf6f6fab67beedbc73bdf75effcfaa9fdd6effa6e9ff7f55edfed3dbfd3bbbca7ede77bfbaeba5fbfa7bbcf7adf3c7f7eaf763fdbfbeade67fefcd5ebb73f")
# core_bytes = bytes.fromhex("05c1490d00300800302b48400ae70284c78602fcbfd60220328b9845dcbb4ba4ea7e4e6655f77b33c076c9b30725564f3d90abf9d0d66b38e83418e5c89ec7d71b430bc29f50e13d63dabc0937718ba9459ff9c4816d3b482f92756e8903cdbedb519672d85141d1f9485a45dfb743e0527794331ee1b15e38316e4fa5896b312fe472abcdb95824cf034a03db2fbfb33249067d2878cad7f0a55e8157473704a7d3e91ac3bcae3caeb4374c18e103")
parser = GzipParser(core_bytes)
bfinal = parser.read_bits(1) # BFINAL
btype = parser.read_bits_rev(2) # BTYPE
hlit = parser.read_bits_rev(5) # HLIT
hdist = parser.read_bits_rev(5) # HDIST
hclen = parser.read_bits_rev(4) # HCLEN
print(bfinal, btype, hlit, hdist, hclen)
code_len_code_lens = [parser.read_bits_rev(3) for _ in range(hclen + 4)]
code_len_code_order = [16, 17, 18, 0, 8, 7, 9, 6, 10, 5, 11, 4, 12, 3, 13, 2, 14, 1, 15]
code_len_code_len_dict = {}
for i in range(hclen + 4):
    if code_len_code_lens[i] > 0:
        code_len_code_len_dict[code_len_code_order[i]] = code_len_code_lens[i]
def produce_huffman_table(code_len: dict[int, int]) -> dict[int, int]:
    max_code_len = max(code_len.values())
    code_len_count = [0] * (max_code_len + 1)
    for c_len in code_len.values():
        if c_len > 0:
            code_len_count[c_len] += 1
    cur_code_start = 0
    code_starts = [0] * (max_code_len + 1)
    for i in range(1, max_code_len + 1):
        cur_code_start = (cur_code_start + code_len_count[i - 1]) << 1
        code_starts[i] = cur_code_start
    res = {}
    for i in sorted(code_len.keys()):
        c_len = code_len[i]
        if c_len > 0:
            res[i] = code_starts[c_len]
            code_starts[c_len] += 1
    return res
max_code_len_code_len = max(code_len_code_len_dict.values())
code_len_codes = produce_huffman_table(code_len_code_len_dict)
code_len_codes_str = {}
for code in code_len_codes.keys():
    code_len_codes_str[code] = f"{code_len_codes[code]:0{code_len_code_len_dict[code]}b}"
print(code_len_codes_str)
def read_code_lens(num_codes: int) -> dict[int, int]:
    res = {}
    i = 0
    prev_code_len = 0
    while i < num_codes:
        x = parser.peek_bits(max_code_len_code_len)
        x_str = f"{x:0{max_code_len_code_len}b}"
        matched_code_len = 0
        matched_bits = 0
        for code_len, code_str in code_len_codes_str.items():
            if x_str.startswith(code_str):
                matched_code_len = code_len
                matched_bits = len(code_str)
                break
        else:
            raise ValueError(f"Invalid code {x_str}")
        parser.advance(matched_bits)
        if matched_code_len < 16:
            res[i] = matched_code_len
            i += 1
            prev_code_len = matched_code_len
        elif matched_code_len == 16:
            repeat = 3 + parser.read_bits_rev(2)
            for _ in range(repeat):
                res[i] = prev_code_len
                i += 1
        elif matched_code_len == 17:
            repeat = 3 + parser.read_bits_rev(3)
            for _ in range(repeat):
                res[i] = 0
                i += 1
            prev_code_len = 0
        elif matched_code_len == 18:
            repeat = 11 + parser.read_bits_rev(7)
            for _ in range(repeat):
                res[i] = 0
                i += 1
            prev_code_len = 0
    return res
lit_huffman_code_len = read_code_lens(hlit + 257)
# print({k: v for k, v in lit_huffman_code_len.items() if v > 0})
dis_huffman_code_len = read_code_lens(hdist + 1)
# print({k: v for k, v in dis_huffman_code_len.items() if v > 0})
lit_huffman_tree = produce_huffman_table(lit_huffman_code_len)
lit_huffman_tree_str = {}
for code in lit_huffman_tree.keys():
    lit_huffman_tree_str[code] = f"{lit_huffman_tree[code]:0{lit_huffman_code_len[code]}b}"
dis_huffman_tree = produce_huffman_table(dis_huffman_code_len)
dis_huffman_tree_str = {}
for code in dis_huffman_tree.keys():
    dis_huffman_tree_str[code] = f"{dis_huffman_tree[code]:0{dis_huffman_code_len[code]}b}"
print(lit_huffman_tree_str)
print(dis_huffman_tree_str)
max_lit_huffman_code_len = max(lit_huffman_code_len.values())
max_dis_huffman_code_len = max(dis_huffman_code_len.values())
print(parser.pos)
res = bytearray()
while True:
    code = parser.peek_bits(max_lit_huffman_code_len)
    code_str = f"{code:0{max_lit_huffman_code_len}b}"
    matched_code = -1
    matched_bits = 0
    for c, c_str in lit_huffman_tree_str.items():
        if code_str.startswith(c_str):
            matched_code = c
            matched_bits = len(c_str)
            break
    else:
        raise ValueError(f"Invalid lit code {code_str}")
    parser.advance(matched_bits)
    if matched_code < 256:
        print(f"Lit: {code_str[:matched_bits]} -> {matched_code}")
        res.append(matched_code)
    elif matched_code == 256:
        # print(f"Fin: {code_str} -> $$")
        break
    else:
        length = matched_code - 257
        # TODO: extra_len
        # extra_len = parser.read_bits_rev(lit_huffman_code_len[matched_code])
        # print(f"Lit: {code_str} {extra_len:0{lit_huffman_code_len[matched_code]}b} -> ({length}, {length + extra_len})")
        assert False, "Not implemented, match code = %d"%matched_code
print(res.decode())

1 2 0 1 14
{1: '100', 6: '101', 7: '110', 16: '0', 17: '1110', 18: '1111'}
{48: '000000', 49: '000001', 50: '000010', 51: '000011', 52: '000100', 53: '000101', 54: '000110', 55: '000111', 56: '001000', 57: '001001', 58: '001010', 59: '001011', 60: '001100', 61: '001101', 62: '001110', 63: '001111', 64: '010000', 65: '010001', 66: '010010', 67: '010011', 68: '010100', 69: '010101', 70: '010110', 71: '010111', 72: '011000', 73: '011001', 74: '011010', 75: '011011', 76: '011100', 77: '011101', 78: '011110', 79: '1111110', 80: '011111', 81: '100000', 82: '100001', 83: '100010', 84: '100011', 85: '100100', 86: '100101', 87: '100110', 88: '100111', 89: '101000', 90: '101001', 91: '101010', 92: '101011', 93: '101100', 94: '101101', 95: '101110', 96: '101111', 97: '110000', 98: '110001', 99: '110010', 100: '110011', 101: '110100', 102: '110101', 103: '110110', 104: '110111', 105: '111000', 106: '111001', 107: '111010', 108: '111011', 109: '111100', 110: '111101', 111: '111110', 256: '1111111'}